In [77]:
import mujoco
from mujoco import viewer
import numpy as np
import plotly.express as px
import contextlib, io
from drone_simulator import DroneSimulator
from pid import PID

model = mujoco.MjModel.from_xml_path("scene.xml")
data = mujoco.MjData(model)
mujoco.mj_forward(model, data)

# viewer = viewer.launch_passive(model, data)
# viewer.cam.distance = 4.
# viewer.cam.lookat = np.array([0, 0, 1])
# viewer.cam.elevation = -30.

desired_altitude = 2

drone_simulator = DroneSimulator(
    model, data, viewer, desired_altitude = desired_altitude,
    altitude_sensor_freq = 0.01,
    wind_change_prob = 0.1,
    rendering_freq = 20,
)

innerPID_freq = 1

In [78]:
innerPID = PID(
    gain_prop = 0.3,
    gain_int = 0.2,
    gain_der = 0,
    sensor_period = drone_simulator.altitude_sensor_period * (0.01 / innerPID_freq),
)

xfac = 0.28
outerPID = PID(
    gain_prop = xfac * 0.1,
    gain_int = 0,#2e-7,
    gain_der = xfac,
    sensor_period = drone_simulator.altitude_sensor_period,
)

In [79]:
fast_ascend_thrust = 3.3
slow_ascend_thrust = 3.25
slow_ascend_acceleration = 0.15
G = 9.81

initial_acc = drone_simulator.data.sensor("body_linacc").data[2] - G
drone_simulator.measured_acceleration = np.array([initial_acc] * 2, dtype = float)

def acceleration_sensor(self):
    self.measured_acceleration[1] = self.measured_acceleration[0]
    self.measured_acceleration[0] = self.data.sensor("body_linacc").data[2] - G
    return self.measured_acceleration

from types import MethodType 
drone_simulator.acceleration_sensor = MethodType(acceleration_sensor, drone_simulator)

In [81]:
thrust = 0
prev_alt_measurement = -1
alts = []

with contextlib.redirect_stdout(io.StringIO()):
    for i in range(4000):
        # TODO: Use the PID controllers in a cascade designe to control the drone

        drone_simulator.acceleration_sensor()

        if drone_simulator.measured_altitudes[0] != prev_alt_measurement:
            wanted_acc = outerPID.output_signal(
                desired_altitude,
                drone_simulator.measured_altitudes
            )

        thrust_addition = innerPID.output_signal(
            wanted_acc,
            drone_simulator.measured_acceleration,
        )

        thrust += thrust_addition

        alts += [drone_simulator.altitude_history[0]]

        drone_simulator.sim_step(thrust, view = False)

alts = np.array(alts[-1500:]) - desired_altitude
error = np.sum(np.abs(alts))
print(error)
# fig = px.line(y=alts, labels={'y':'altitude'})
# fig.show()


8.99370597205164
